# DMU-Net Dataset Generation Notebook

* **Creator:** Jonathan DEKHTIAR
* **Date:** 2017-05-21
<br/><br/>
* **Contact:** [contact@jonathandekhtiar.eu](mailto:contact@jonathandekhtiar.eu)
* **Twitter:** [@born2data](https://twitter.com/born2data)
* **LinkedIn:** [JonathanDEKHTIAR](https://fr.linkedin.com/in/jonathandekhtiar)
* **Personal Website:** [JonathanDEKHTIAR](http://www.jonathandekhtiar.eu)
* **RSS Feed:** [FeedCrunch.io](https://www.feedcrunch.io/@dataradar/)
* **Tech. Blog:** [born2data.com](http://www.born2data.com/)
* **Github:** [DEKHTIARJonathan](https://github.com/DEKHTIARJonathan)
<br/><br/>

```
*************************************************************************
**
** 2017 March 13
**
** In place of a legal notice, here is a blessing:
**
**    May you do good and not evil.
**    May you find forgiveness for yourself and forgive others.
**    May you share freely, never taking more than you give.
**
*************************************************************************
```

## Objectives

This notebook generates a [TFRecords](https://www.tensorflow.org/api_guides/python/python_io#TFRecords_Format_Details) file from the images contained in the "./data/" folder.

This will be used later to retrain an CNN model: [Inception-V3](http://arxiv.org/abs/1512.00567) model developed by Szegedy et al. The model has been Pre-Trained with the [ImageNet](http://www.image-net.org/) dataset allowing a much more accurate result due to the large number of data avaiable in this dataset. We call this a "Transfer Learning".

## 1. Load the necessary libraries and initialise global variables

In [1]:
import os

import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt

%matplotlib inline

####################### GLOBAL Notebook

NUM_EPOCH          = 1
INPUT_DIRECTORY    = "data"
TFRECORD_FILENAME  = "output/dmu_net.tfrecord"
SEED               = 666 # A specific number for reproducability or None for a random value

# Model Dependant Parameters - Inception V3

IMG_HEIGHT         = 299     # This parameter is fixed due to the model used: Inception-V3
IMG_WIDTH          = 299     # This parameter is fixed due to the model used: Inception-V3
IMG_CHANNELS       = 3       # This parameter is fixed due to the model used: Inception-V3
IMG_COLORSPACE     = "RGB"   # This parameter is fixed due to the model used: Inception-V3
IMG_FORMAT         = "JPEG"  # This parameter is fixed due to the model used: Inception-V3

## 2. File Queue and Image Reading Process Definition

### 2.1 Define a queue of all the images in "jpeg" in the specific data folder

Make a queue of file names including all the JPEG images files in the relative image directory.

In [2]:
data_directories = [ name for name in os.listdir(INPUT_DIRECTORY) if os.path.isdir(os.path.join(INPUT_DIRECTORY, name)) ]

all_files = [tf.train.match_filenames_once(INPUT_DIRECTORY + "/" + x + "/*.jpg") for x in data_directories]

filename_queue = tf.train.string_input_producer(
    tf.concat(all_files,0), # Merge the sub-tensors into one
    num_epochs=NUM_EPOCH,
    seed=SEED,
    shuffle=True
)

### 2.2. Define the image reader

Read an entire image file which is required since they're JPEGs, if the images are too large they could be split in advance to smaller files or use the Fixed reader to split up the file.

In [3]:
image_reader = tf.WholeFileReader()

### 2.3. Read images from the Queue One by One
Read a whole file from the queue, the first returned value in the tuple is the filename which we are ignoring.

In [4]:
image_path, image_file = image_reader.read(filename_queue)

### 2.4. Convert each Image to a Tensor

Decode the image file, this will turn it into a Tensor which we can then use in training. It automatically detect whether the image is ["GIF", "PNG", "JPEG"] and which decoder to use.

In [5]:
def string_length_tf(t):
    return tf.py_func(lambda x: len(x), [t], tf.int32)

In [6]:
path_length = string_length_tf(image_path)
file_extension = tf.substr(image_path, path_length - 3, 3)

file_cond = tf.equal(file_extension, 'jpg')
        
image_tmp      = tf.cond(
                    file_cond, 
                    lambda: tf.image.decode_jpeg(image_file), 
                    lambda: tf.image.decode_png(image_file)
               )

image_resized  = tf.image.resize_images(
                    image_tmp, 
                    tf.stack([IMG_HEIGHT, IMG_WIDTH]), 
                    method=tf.image.ResizeMethod.BICUBIC,
                    align_corners=True
               )

# resize image by bilinear, bicubic and area will change image data type(from uint8 to float32)
image_data = tf.cast(image_resized, tf.uint8) # We need to convert it back to unint8 to display it properly

image_shape    = tf.shape(image_data)
image_height   = image_shape[0]
image_width    = image_shape[1]
image_channels = image_shape[2]

image_label    = tf.string_split([image_path] , delimiter=os.path.sep).values[1]

## 3. Perform Image Augmentation

### 3.1 Define an Image Augmentation Function

In [7]:
def augment_image(image):
    # randomly shift gamma
    random_gamma      = tf.random_uniform([], 0.8, 1.2)
    image_aug         = image ** random_gamma

    # randomly shift brightness
    random_brightness = tf.random_uniform([], 0.5, 1.2)
    image_aug         =  image_aug * random_brightness
    
    '''
    # randomly shift color
    random_colors     = tf.random_uniform([3], 0.8, 1.2)
    
    white             = tf.ones([tf.shape(image)[0], tf.shape(image)[1]])    
    color_image       = tf.stack([white * random_colors[i] for i in range(3)], axis=2)
    
    image_aug         *= color_image
    '''
    
    # saturate
    #image_aug         = tf.clip_by_value(image_aug,  0, 255)
    
    brightest_val = tf.reduce_max(image_aug)
    
    #image_aug = 
    image_aug = tf.cond(
        brightest_val > 255, 
        lambda: image_aug * (255.0 / brightest_val), 
        lambda: image_aug
    )
    
    # randomly horizontally flip the image
    do_flip = tf.random_uniform([], 0, 1)
    image_aug  = tf.cond(do_flip > 0.5, lambda: tf.image.flip_left_right(image_aug), lambda: image_aug)
    
    # randomly rotate the image
    n_rot = tf.random_uniform([], 0, 3, tf.int32) # 0 => No Rotation, 1 => 90° Rot, 2 => 180° Rot, 3 => 270° Rotation
    image_aug = tf.image.rot90(image_aug, n_rot)
    
    #Convert to UINT-8 and Return
    return tf.cast(image_aug, tf.uint8)

### 3.2. Create a Tensor of Images

In [8]:
img_arr = tf.stack([
    image_data,
])

for _ in range(30):
    img_arr = tf.concat([img_arr, [augment_image(image_resized)]], 0)

## 3. Defining the TFRecord Saving Process

### 3.1. Defining the tf.train.Features functions

In [9]:
def _float_feature(value):
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

### 3.2. Defining the writer function

In [19]:
def writer_to_tfrecord(height, width, colorspace, channels, label, text, image_format, filename, image_buffer):
    
    # write label, shape, and image content to the TFRecord file
    example = tf.train.Example(features=tf.train.Features(feature={    
        'image/height'      : _int64_feature(height),
        'image/width'       : _int64_feature(width),
        'image/colorspace'  : _bytes_feature(tf.compat.as_bytes(colorspace)),
        'image/channels'    : _int64_feature(channels),
        'image/class/label' : _int64_feature(label),
        'image/class/text'  : _bytes_feature(tf.compat.as_bytes(text)),
        'image/format'      : _bytes_feature(tf.compat.as_bytes(image_format)),
        'image/filename'    : _bytes_feature(tf.compat.as_bytes(os.path.basename(filename))),
        'image/encoded'     : _bytes_feature(tf.compat.as_bytes(image_buffer.tobytes()))
    }))
    
    writer.write(example.SerializeToString())

## 4. Define an Initialisation Operation

In [20]:
init_op_global = tf.global_variables_initializer()
init_op_local = tf.local_variables_initializer()

## 5. Launch the dataset generation Session

In [22]:
with tf.Session() as sess:
    sess.run([init_op_global, init_op_local])

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    writer = tf.python_io.TFRecordWriter(TFRECORD_FILENAME)
    
    try:
        i = 1
        while not coord.should_stop():
            
            _h, _w, _chn, _lbl_txt, _img_pth, _img_arr = sess.run([
                image_height, 
                image_width, 
                image_channels, 
                image_label, 
                image_path,
                img_arr
            ])
            
            _lbl_idx = data_directories.index(str(_lbl_txt,'utf-8'))
            
            '''
            if i <= 1:
                print("height:", _h)
                print("width:", _w)
                print("channels:", _chn)
                print("label:", _lbl_txt)
                
                for img in _img_arr:
                    plt.imshow(img)
                    plt.show()

                print()
            '''
            
            for _img in _img_arr:
                writer_to_tfrecord(
                    height        = _h, 
                    width         = _w, 
                    colorspace    = IMG_COLORSPACE, 
                    channels      = IMG_CHANNELS, 
                    label         = _lbl_idx, 
                    text          = _lbl_txt, 
                    image_format  = IMG_FORMAT, 
                    filename      = _img_pth, 
                    image_buffer  = _img
                )
            
            if (i % 300 == 0):
                print ("Processing Image:", i)
                
            i += 1
            
    except tf.errors.OutOfRangeError:
        pass
    
    finally:        
        print("\nNumber of Images Processed:", i)
        
        writer.close()
        coord.request_stop()
        coord.join(threads)

Processing Image: 300
Processing Image: 600
Processing Image: 900
Processing Image: 1200
Processing Image: 1500
Processing Image: 1800
Processing Image: 2100
Processing Image: 2400
Processing Image: 2700
Processing Image: 3000
Processing Image: 3300
Processing Image: 3600

Number of Images Processed: 3671
